# Prédiction d'age et de sexe à partir de voix humaine

## Introduction

Dans ce notebook, nous allons essayer de prédire l'âge et le sexe d'une personne à partir de sa voix. Pour cela, nous allons utiliser un ensemble de données de voix humaine disponible sur [Commonvoice](https://commonvoice.mozilla.org/fr/datasets ). Cet ensemble de données contient des enregistrements vocaux de personnes, chacun étant associé à un label indiquant le sexe de la personne (homme ou femme) et son âge. Nous allons entraîner un modèle de deeplearning sur ces données pour prédire le sexe et l'âge d'une personne à partir de sa voix.

1. [Paramètres](#Paramètres)
2. [Préparation des données](#Preparation-des-donnees)
3. [Traitements des audios](#Traitements-des-audios)
4. [Création du modèle](#Création-du-modèle)
5. [Entrainer le modèle](#4)
6. [Résultats](#5)




## Paramètres

On a une liste de paramètres qui vont nous permettre de configurer le modèle, les données et l'entraînement.

In [ ]:
train_age_only = True
use_ordinal_age = False
use_early_stopping = False
features_length = 40

# Hyperparameters
loss_age_weight = 1
loss_genre_weight = 1
learning_rate = 0.1
epochs = 10
batch_size = 64
num_age_classes = 7
train_ratio = 0.90

## Préparation des données

Le dataset choisis comportait énormément de lignes nous avons donc décidé dans un premier temps de supprimer toutes les lignes ou une valeur manquait. Puisque nous voulons prédire l'âge et le sexe nous avons ensuite trier le dataset pour voir la proportion de chaque sexe par tranche d'âge. A partir de ce résultat nous avons décidé de garder 10_000 lignes pour chaque catégorie d'âge avec une répartition égale d'hommes et de femmes.

In [ ]:
data_path = r"D:\datasets\cv-corpus-19.0-2024-09-13-fr\cv-corpus-19.0-2024-09-13\fr"
data_path = r"/mnt/d/datasets/cv-corpus-19.0-2024-09-13-fr/cv-corpus-19.0-2024-09-13/fr"

In [ ]:
import pandas as pd

data = pd.read_table(f"{data_path}/validated.tsv")

keep_columns = ['path', 'sentence','age', 'gender',]
export_df = data.copy()[keep_columns]

#filter out the rows with missing values
import math
print(export_df.isnull().sum()/len(export_df)*100)
print("with na : ", export_df.shape)
print("without na : ",export_df.dropna().shape)
print(f"percent of data lost : {math.floor((export_df.shape[0] - export_df.dropna().shape[0])/export_df.shape[0]*100)}%")
filtered_df = export_df.copy().dropna()
filtered_df = filtered_df[filtered_df['gender'] != "non-binary"]
filtered_df = filtered_df[filtered_df['gender'] != "do_not_wish_to_say"]


# plot the distribution of age by gender
import matplotlib.pyplot as plt
import seaborn as sns
def show_distrib(df):
    plt.figure(figsize=(10, 6))
    ax = sns.histplot(data=df, x='age', hue='gender', multiple='stack', stat='count')

    # Annotate the bars with their counts
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    plt.title('Distribution of Age by Gender')
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.show()

show_distrib(filtered_df)


# export a csv with X rows by age category with the same distribution of men and woman
age_df_list = []
for age in filtered_df['age'].unique():
    df_age = filtered_df[filtered_df['age'] == age]
    df_male = df_age[df_age['gender'] == 'male_masculine']
    df_female = df_age[df_age['gender'] == 'female_feminine']
    count_per_gender = min(row_per_age, len(df_female))
    df_tmp = pd.concat([df_female[:count_per_gender], df_male[:count_per_gender]])
    age_df_list.append(df_tmp)
export_df = pd.concat(age_df_list)
export_df.head()

show_distrib(export_df)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\datasets\\cv-corpus-19.0-2024-09-13-fr\\cv-corpus-19.0-2024-09-13\\fr/validated.tsv'

## Traitements des audios

Nous étions d'abord parti pour utiliser le spectrogramme des audios en normalisant l'audio et en modifiant la durée à 10 secondes. Cependant, les résultats obtenus n'étaient pas pertinent (~0.23 d'accuracy) nous avons donc décidé de changer de méthode. Nous avons utilisé la librairie librosa pour transformer les audios en MFCC (Mel-Frequency Cepstral Coefficients) qui sont des caractéristiques audio qui représentent le spectre de puissance d'un son, utilisées pour l'analyse et la reconnaissance vocale. Nous avons aussi fait la moyenne des MFCC pour chaque audio pour réduire la dimension des données.

In [ ]:
import librosa
import matplotlib.pyplot as plt
import time
import numpy as np
import os
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

def load_wav(file_path: str):
    """
    Load a wav file using librosa
    :param file_path: Path to the wav file
    :return: Tuple of the audio data and the sample rate
    """
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    return audio_data, sample_rate


def get_spectrogram(audio):
    # Short-time Fourier transform (STFT).
    stft = abs(librosa.stft(audio))

    # Convert an amplitude spectrogram to dB-scaled spectrogram.
    spectrogram = librosa.amplitude_to_db(stft)

    return spectrogram


def get_mfcc(audio, sampling_rate, n_mfcc=20):
    mfcc = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=n_mfcc)
    return mfcc


def clip_audio(audio, duration=10, repeat=False):
    """
    Clip the audio to a specific duration
    :param audio: Audio data
    :param duration: Duration to clip the audio to
    :return: Clipped audio data
    """
    # Get the number of samples for the desired duration
    n_samples = int(duration * sample_rate)
    if len(audio) < n_samples:
        # Repeat the audio if it is shorter than the desired duration
        if repeat:
            n_repeats = n_samples // len(audio)
            n_samples = n_repeats * len(audio)
            audio = np.tile(audio, n_repeats)
        else:
            # Pad the audio with zeros if it is shorter than the desired duration
            n_zeros = n_samples - len(audio)
            audio = np.pad(audio, (0, n_zeros))
    # Clip the audio
    clipped_audio = audio[:n_samples]
    return clipped_audio


def extract_features(file_name, base_path, output_path, save=False):
    audio_data, sample_rate = load_wav(os.path.join(base_path, file_name))
    spectrogram = get_mfcc(audio_data, sample_rate, n_mfcc=40)
    features = []
    for el in spectrogram:
        features.append(np.mean(el))
    features = np.array(features)
    if save:
        np.save(os.path.join(output_path, file_name.split(".")[0]), features)
    return file_name, features


if __name__ == "__main__":
    # apply_async example
    base_path = r"/mnt/d/datasets/cv-corpus-19.0-2024-09-13-fr/cv-corpus-19.0-2024-09-13/fr/clips/"
    start_time = time.time()
    num_processes = 8
    data = pd.read_csv(os.path.join("../","validated_filtered_500_per_age.csv"), sep=',')
    paths = data['path'].to_list()
    # paths = paths[:8]
    print(f"Number of paths: {len(paths)}")
    with Pool(num_processes) as p:
        async_result = p.starmap_async(extract_features, [(path, base_path, "processed") for path in paths])
        print('Waiting for results...')
        results = async_result.get()
    print(f"Multi --- {time.time() - start_time} seconds ---")
    df = pd.DataFrame(results, columns=["path", "mfcc_features"])
    # merge the features with the data
    data = data.merge(df, left_on='path', right_on='path')
    data.to_csv("features.csv", index=False)

## Création du dataset

Nous avons créé un dataset avec les MFCC moyens de chaque audio et les labels associés. Nous avons ensuite divisé ce dataset en un ensemble d'entraînement et un ensemble de test. On utilise Tensorflow pour charger les données et les préparer pour l'entraînement.

Pour encoder le genre nous avons choisi un encodage binaire 0 (male) et 1 (femelle).
Pour l'âge nous avons essayé plusieurs encodages : 
- l'encodage ordinnal semble être une option logique pour l'âge, mais les résultats n'étaient pas ceux espérés .
- Le hot-one encoding a donné de meilleurs résultats.

In [ ]:
from sklearn.model_selection import train_test_split


def preprocess_label_data(label_data, keep_age=False):
    gender_dict = {
        'male_masculine': 0,
        'female_feminine': 1
    }
    age_dict = {
        'teens': 0,
        'twenties': 1,
        'thirties': 2,
        'fourties': 3,
        'fifties': 4,
        'sixties': 5,
        'seventies': 6
    }
    label_data = label_data.copy()
    mfcc_features = []
    for i in range(features_length):
        mfcc_features.append(label_data[f'mfcc_{i}'])
        label_data = label_data.drop(f'mfcc_{i}', axis=1)
    label_data['mfcc_features'] = list(np.asarray(mfcc_features).T)
    if keep_age:
        serie_age = label_data['age'].copy()
    if use_ordinal_age:
        label_data['age'] = label_data['age'].map(age_dict)
    else:
        label_data = pd.get_dummies(label_data, columns=['age'])
    label_data['gender'] = label_data['gender'].map(gender_dict)
    return pd.concat([label_data, serie_age], axis=1) if keep_age else label_data

Pour la création du datasets nous avons plusieurs étapes :
- Charger les données.
- Encoder les labels.
- Diviser les données en un ensemble d'entraînement et un ensemble de test.
- Créer des datasets TensorFlow à partir des données.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf


def load_csv_data(csv_path):
    """
    Load CSV data into a DataFrame with specific columns.
    """
    df = pd.read_csv(csv_path)
    return df


def create_dataset(csv_path, batch_size, num_age_classes, train_ratio=0.8, random_state=0):
    """
    Create a tf.data.Dataset from a CSV file.
    """
    # Load CSV and preprocess
    df = load_csv_data(csv_path)
    df = preprocess_label_data(df)

    # Split the data
    train_data, val_data = train_test_split(df, train_size=train_ratio, random_state=random_state,
                                            stratify=df['gender'])
    print(f"Train data: {len(train_data)} samples")
    print(f"Validation data: {len(val_data)} samples")
    # Convert columns to tensors
    def convert_to_tensors(data):
        features = data['mfcc_features']
        features = tf.convert_to_tensor(np.stack(features), dtype=tf.float32)
        ages = tf.convert_to_tensor(data['age'] if use_ordinal_age else data.iloc[:, 4:].values,
                                    dtype=tf.float32)  # Assuming age columns start from index 3
        if not train_age_only:
            genders = tf.convert_to_tensor(data['gender'].values, dtype=tf.int32)
            return features, genders, ages
        else:
            return features, ages

    if not train_age_only:
        train_features, train_genders, train_ages = convert_to_tensors(train_data)
        val_features, val_genders, val_ages = convert_to_tensors(val_data)
        # Create datasets from tensors
        train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_genders, train_ages))
        val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_genders, val_ages))

        # Parse rows, and batch
        train_dataset = (
            train_dataset
            .map(lambda path, gender, age: tf_parse_row(path, gender, age, num_age_classes),
                 num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE)  # Prefetch for efficient data loading
        )

        val_dataset = (
            val_dataset
            .map(lambda path, gender, age: tf_parse_row(path, gender, age, num_age_classes),
                 num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE)  # Prefetch for efficient data loading
        )
    else:
        train_features, train_ages = convert_to_tensors(train_data)
        val_features, val_ages = convert_to_tensors(val_data)
        # Create datasets from tensors
        train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_ages))
        val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_ages))

        # Parse rows, and batch
        train_dataset = (
            train_dataset
            .map(lambda features, age: tf_parse_row(features, None, age, num_age_classes),
                 num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE)  # Prefetch for efficient data loading
        )

        val_dataset = (
            val_dataset
            .map(lambda features, age: tf_parse_row(features, None, age, num_age_classes),
                 num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE)  # Prefetch for efficient data loading
        )
    return train_dataset, val_dataset


def tf_parse_row(features, gender_label, age_label, num_age_classes):
    """
    Wrapper to use the parse_row function with TensorFlow.
    """

    # Set shapes for TensorFlow to understand
    features.set_shape([features_length])  # Ajustez cette forme en fonction de vos données
    age_label.set_shape([] if use_ordinal_age else [num_age_classes])
    labels = {"age": age_label}
    if not train_age_only:
        gender_label.set_shape([])
        labels['gender'] = gender_label
    return features, labels



## Création du modèle

Nous avons voulu reconnaitre des patternes dans les audios pour cela on a choisi des couches de convolution qui sont très utilisées pour la reconnaissance de patternes. Nous avons combiné ces couches avec des couches de maxpooling pour réduire la dimension des données et des couches de dropout pour éviter l'overfitting. 
Des couches de dense ont été ajoutées pour la classification des différents patterns reconnus. 

Pour les fonctions de pertes nous avons utilisé la Binary Crossentropy pour le sexe et la Categorical Crossentropy pour l'age. Nous avons aussi jouer sur le poid de chaque fonction de perte pour guider le modèle dans son apprentissage.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, initializers


class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='same', activation='relu',
                 kernel_initializer='he_normal', batch_norm=True, max_pool=True, dropout_rate=0.0):
        super(ConvBlock, self).__init__()
        self.conv1 = layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            kernel_initializer=kernel_initializer
        )
        self.conv2 = layers.Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            kernel_initializer=kernel_initializer
        )
        self.batch_norm = layers.BatchNormalization() if batch_norm else None
        self.activation = layers.Activation(activation)
        self.max_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same') if max_pool else None
        self.dropout = layers.Dropout(dropout_rate) if dropout_rate > 0 else None

    def call(self, inputs, training=False):
        x = self.conv1(inputs, training=training)
        x = self.conv2(x, training=training)
        if self.batch_norm:
            x = self.batch_norm(x, training=training)
        x = self.activation(x)
        if self.max_pool:
            x = self.max_pool(x)
        if self.dropout:
            x = self.dropout(x, training=training)
        return x


class AudioAgeAndGenderClassifier(tf.keras.Model):
    def __init__(self, input_shape=40):
        super(AudioAgeAndGenderClassifier, self).__init__()
        self.blocks = [
            ConvBlock(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu',
                      kernel_initializer='he_normal', batch_norm=False, max_pool=True, dropout_rate=0.0),
            ConvBlock(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu',
                      kernel_initializer='he_normal', batch_norm=False, max_pool=True, dropout_rate=0.0),
            ConvBlock(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu',
                      kernel_initializer='he_normal', batch_norm=False, max_pool=True, dropout_rate=0.0),
            ConvBlock(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu',
                      kernel_initializer='he_normal', batch_norm=False, max_pool=True, dropout_rate=0.0),
            # ConvBlock(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu',
            #           kernel_initializer='he_normal', batch_norm=True, max_pool=True, dropout_rate=0.5),
        ]

        self.reduction_layer = layers.Flatten()
        self.dense = layers.Dense(128, activation='sigmoid')
        self.age_output = layers.Dense(1 if use_ordinal_age else num_age_classes,
                                       activation="relu" if use_ordinal_age else 'softmax')
        if not train_age_only:
            self.gender_output = layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = tf.expand_dims(inputs, axis=-1)
        # x = inputs
        for block in self.blocks:
            x = block(x, training=training)

        x = self.reduction_layer(x)
        x = self.dense(x, training=training)
        age_pred = self.age_output(x)

        if not train_age_only:
            gender_pred = self.gender_output(x)
            return {"age": age_pred, "gender": gender_pred}
        else:
            return {"age": age_pred}


# Instantiate the model
device_name = tf.test.gpu_device_name()
if device_name:
    print(f"Using GPU: {device_name}")
else:
    print("Using CPU")

KeyboardInterrupt: 

## Entrainer le modèle

Une fois le modèle définit on peut l'entrainer. Nous avons fait  plusieurs entrainments en faisant varier les paramètres pour comparer l'influence de ces derniers sur les résultats. Pour pouvoir finalement trouver les meilleurs paramètres pour notre modèle.

L'âge étant un label plus complexe à prédire que le genre on peut choisir d'entrainer le modèle que sur l'âge pour mieux régler les hyperparamètres.

In [ ]:
# Train the model
def train_model(model, dataset, validation_dataset, epochs, batch_size, verbose=0):
    # Define callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            "best_model.keras", save_best_only=True, monitor="val_loss"
        ),
        tf.keras.callbacks.TensorBoard(log_dir="./logs"),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=lambda epoch, logs: print_example_output(model, val_dataset)
        ),
    ]
    if use_early_stopping:
        callbacks.append(
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=3
            ), )

    # Train the model
    history = model.fit(
        dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=verbose,
    )
    return history


csv_path = f"{data_path}/validated_filtered_100_per_age.csv"

# csv_path = f"{data_path}/validated_filtered_100.csv"
csv_path = f"{data_path}/features.csv"

train_dataset, val_dataset = create_dataset(csv_path, batch_size, num_age_classes, train_ratio)

## Résultats

#TODO penser à mettre en gras les meilleurs résultats de chaques colonnes